### To Do / License





- Create a materialize view of a campaign (all fields needed and sales and such)
  - Generate this data if we do not have it (we need segments, sales data, coupons, channels, etc)
- Run Data Insight on the materialized view
- Call the REST API to get the SQL generated from Data Insights
- Run the SQL through Gemini to create English descriptions (NOT REQUIRED)
- The demo app will call this and run the SQL

```
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
```

Author: Adam Paternostro

### Pip installs

In [ ]:
# To read/write to/from Kafka
import sys

# https://PLACEHOLDER.com/index.html
# !{sys.executable} -m pip install PLACEHOLDER

### Initialize

In [ ]:
from PIL import Image
from IPython.display import HTML
from IPython.display import Audio
from functools import reduce
import IPython.display
import google.auth
import requests
import json
import uuid
import base64
import os
import cv2
import random
import time
import datetime
import base64
import random

In [ ]:
# Set these (run this cell to verify the output)

bigquery_location = "us"
region = "us-central1"

# Get some values using gcloud
project_id = !(gcloud config get-value project)
user = !(gcloud auth list --filter=status:ACTIVE --format="value(account)")

if len(project_id) != 1:
  raise RuntimeError(f"project_id is not set: {project_id}")
project_id = project_id[0]

if len(user) != 1:
  raise RuntimeError(f"user is not set: {user}")
user = user[0]

print(f"project_id = {project_id}")
print(f"user = {user}")

### Helper Methods

#### restAPIHelper
Calls the Google Cloud REST API using the current users credentials.

In [ ]:
def restAPIHelper(url: str, http_verb: str, request_body: str) -> str:
  """Calls the Google Cloud REST API passing in the current users credentials"""

  import requests
  import google.auth
  import json

  # Get an access token based upon the current user
  creds, project = google.auth.default()
  auth_req = google.auth.transport.requests.Request()
  creds.refresh(auth_req)
  access_token=creds.token

  headers = {
    "Content-Type" : "application/json",
    "Authorization" : "Bearer " + access_token
  }

  if http_verb == "GET":
    response = requests.get(url, headers=headers)
  elif http_verb == "POST":
    response = requests.post(url, json=request_body, headers=headers)
  elif http_verb == "PUT":
    response = requests.put(url, json=request_body, headers=headers)
  elif http_verb == "PATCH":
    response = requests.patch(url, json=request_body, headers=headers)
  elif http_verb == "DELETE":
    response = requests.delete(url, headers=headers)
  else:
    raise RuntimeError(f"Unknown HTTP verb: {http_verb}")

  if response.status_code == 200:
    return json.loads(response.content)
    #image_data = json.loads(response.content)["predictions"][0]["bytesBase64Encoded"]
  else:
    error = f"Error restAPIHelper -> ' Status: '{response.status_code}' Text: '{response.text}'"
    raise RuntimeError(error)

#### Gemini Pro

In [ ]:
def GeminiLLM(prompt, model = "gemini-1.5-pro-001", response_schema = None,
                 temperature = 1, topP = 1, topK = 32):

  # https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/inference#supported_models
  # model = "gemini-1.5-pro-001"
  # model = "gemini-pro" # This does support topK (but the function is more generic)
  # model = "gemini-1.0-pro" # This does not support response_schema

  llm_response = None
  if temperature < 0:
    temperature = 0

  creds, project = google.auth.default()
  auth_req = google.auth.transport.requests.Request() # required to acess access token
  creds.refresh(auth_req)
  access_token=creds.token

  headers = {
      "Content-Type" : "application/json",
      "Authorization" : "Bearer " + access_token
  }

  # https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/inference
  url = f"https://{location}-aiplatform.googleapis.com/v1/projects/{project_id}/locations/{location}/publishers/google/models/{model}:generateContent"

  generation_config = {
    "temperature": temperature,
    "topP": topP,
    "maxOutputTokens": 8192,
    "candidateCount": 1,
    "responseMimeType": "application/json",
  }

  # Add inthe response schema for when it is provided
  if response_schema is not None:
    generation_config["responseSchema"] = response_schema

  if model == "gemini-pro" or model == "gemini-1.0-pro" or model == "gemini-1.0-pro-vision-001":
    generation_config["topK"] = topK

  payload = {
    "contents": {
      "role": "user",
      "parts": {
          "text": prompt
      },
    },
    "generation_config": {
      **generation_config
    },
    "safety_settings": {
      "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
      "threshold": "BLOCK_LOW_AND_ABOVE"
    }
  }

  response = requests.post(url, json=payload, headers=headers)

  if response.status_code == 200:
    try:
      json_response = json.loads(response.content)
    except Exception as error:
      raise RuntimeError(f"An error occurred parsing the JSON: {error}")

    if "candidates" in json_response:
      candidates = json_response["candidates"]
      if len(candidates) > 0:
        candidate = candidates[0]
        if "content" in candidate:
          content = candidate["content"]
          if "parts" in content:
            parts = content["parts"]
            if len(parts):
              part = parts[0]
              if "text" in part:
                text = part["text"]
                llm_response = text
              else:
                raise RuntimeError("No text in part: {response.content}")
            else:
              raise RuntimeError("No parts in content: {response.content}")
          else:
            raise RuntimeError("No parts in content: {response.content}")
        else:
          raise RuntimeError("No content in candidate: {response.content}")
      else:
        raise RuntimeError("No candidates: {response.content}")
    else:
      raise RuntimeError("No candidates: {response.content}")

    # Remove some typically response characters (if asking for a JSON reply)
    llm_response = llm_response.replace("```json","")
    llm_response = llm_response.replace("```","")
    llm_response = llm_response.replace("\n","")

    return llm_response

  else:
    raise RuntimeError(f"Error with prompt:'{prompt}'  Status:'{response.status_code}' Text:'{response.text}'")

#### Helper Functions

In [ ]:
def RunQuery(sql):
  import time
  from google.cloud import bigquery
  client = bigquery.Client()

  if (sql.startswith("SELECT") or sql.startswith("WITH")):
      df_result = client.query(sql).to_dataframe()
      return df_result
  else:
    job_config = bigquery.QueryJobConfig(priority=bigquery.QueryPriority.INTERACTIVE)
    query_job = client.query(sql, job_config=job_config)

    # Check on the progress by getting the job's updated state.
    query_job = client.get_job(
        query_job.job_id, location=query_job.location
    )
    print("Job {} is currently in state {} with error result of {}".format(query_job.job_id, query_job.state, query_job.error_result))

    while query_job.state != "DONE":
      time.sleep(2)
      query_job = client.get_job(
          query_job.job_id, location=query_job.location
          )
      print("Job {} is currently in state {} with error result of {}".format(query_job.job_id, query_job.state, query_job.error_result))

    if query_job.error_result == None:
      return True
    else:
      raise Exception(query_job.error_result)

In [ ]:
# Since our Primary keys are INTs we get the next available value
def GetNextPrimaryKey(fully_qualified_table_name, field_name):
  import time
  from google.cloud import bigquery
  client = bigquery.Client()

  sql = f"""
  SELECT IFNULL(MAX({field_name}),0) AS result
    FROM `{fully_qualified_table_name}`
  """
  # print(sql)
  df_result = client.query(sql).to_dataframe()
  # display(df_result)
  return df_result['result'].iloc[0] + 1

In [ ]:
def PrettyPrintJson(json_string):
  json_object = json.loads(json_string)
  json_formatted_str = json.dumps(json_object, indent=2)
  print(json_formatted_str)
  return json.dumps(json_object)

### Main Code

#### Create BigQuery table to hold results

In [ ]:
%%bigquery

--DROP TABLE IF EXISTS `chocolate_ai.data_insights`;

CREATE TABLE IF NOT EXISTS `chocolate_ai.data_insights`
(
    data_insights_id                  INTEGER NOT NULL OPTIONS(description="Primary key."),
    data_insights_scan_name           STRING  NOT NULL OPTIONS(description="The name of the data insights scan."),
    data_insights_dataset_name        STRING  NOT NULL OPTIONS(description="The name of the data insights dataset."),
    data_insights_table_name          STRING  NOT NULL OPTIONS(description="The name of the data insights table."),

    data_insights_sql                 STRING  NOT NULL OPTIONS(description="The generated SQL by data insights"),
    data_insights_sql_description     STRING  NOT NULL OPTIONS(description="The generated Description by data insights"),
)
CLUSTER BY data_insights_scan_name;

#### createDataDocumentScan

In [ ]:
def createDataDocumentScan(data_insights_scan_name, data_insights_display_name, data_insights_dataset_name, data_insights_data_scan_table_name):
  """Tests to see if the Document (not a data profile or data quality) scan is created and if not, creates it."""

  """
  !curl \
    'https://dataplex.googleapis.com/v1/projects/data-analytics-preview/locations/us-central1/dataScans' \
    --header "Authorization: Bearer $(gcloud auth print-access-token)" \
    --header 'Accept: application/json' \
    --compressed
  {
        "name": "projects/data-analytics-preview/locations/us-central1/dataScans/chocolate-ai-looker-sales-data-docuemnt-scan",
        "uid": "4654c390-a841-4939-bd0b-8ba2d151bd03",
        "displayName": "Chocolate A.I. - Looker Sales Data Documenation Scan",
        "state": "ACTIVE",
        "createTime": "2024-08-22T18:07:50.455838572Z",
        "updateTime": "2024-08-22T18:07:55.294013073Z",
        "data": {
          "resource": "//bigquery.googleapis.com/projects/data-analytics-preview/datasets/chocolate_ai/tables/looker_sales_data"
        },
        "executionSpec": {
          "trigger": {
            "onDemand": {}
          }
        },
        "executionStatus": {},
        "type": "DATA_DOCUMENTATION"
      },
  """

  # Gather existing data scans
  # https://cloud.google.com/dataplex/docs/reference/rest/v1/projects.locations.dataScans/list

  url = f"https://dataplex.googleapis.com/v1/projects/{project_id}/locations/{region}/dataScans"

  # Gather existing data scans
  json_result = restAPIHelper(url, "GET", None)
  print(f"createDataDocumentScan (GET) json_result: {json_result}")

  # Test to see if data scan exists, if so return
  if "dataScans" in json_result:
    for item in json_result["dataScans"]:
      print(f"Scan names: {item['name']}")
      # "projects/data-analytics-preview/locations/us-central1/clusters/kafka-cluster"
      if item["name"] == f"projects/{project_id}/locations/{region}/dataScans/{data_insights_scan_name}":
        print(f"Data Document Scan {data_insights_scan_name} already exists")
        return f"projects/{project_id}/locations/{region}/dataScans/{data_insights_scan_name}"

  """
  # Create a Documentation Scan
  # Create
  #
  !curl --request POST \
    'https://dataplex.googleapis.com/v1/projects/data-analytics-preview/locations/us-central1/dataScans?dataScanId=chocolate-ai-looker-sales-data-docuemnt-scan' \
    --header "Authorization: Bearer $(gcloud auth print-access-token)" \
    --header 'Accept: application/json' \
    --header 'Content-Type: application/json' \
    --data '{"displayName":"Chocolate A.I. - Looker Sales Data Documenation Scan","type": "DATA_DOCUMENTATION",  "dataDocumentationSpec": {}, "data":{"resource":"//bigquery.googleapis.com/projects/data-analytics-preview/datasets/chocolate_ai/tables/looker_sales_data"}}' \
    --compressed
  """

  # Create a new scan
  # https://cloud.google.com/dataplex/docs/reference/rest/v1/projects.locations.dataScans/create
  print("Creating Document Data Scan")

  url = f"https://dataplex.googleapis.com/v1/projects/{project_id}/locations/{region}/dataScans?dataScanId={data_insights_scan_name}"

  request_body = {
      "displayName": data_insights_display_name,
      "type": "DATA_DOCUMENTATION",
      "dataDocumentationSpec": {},
      "data":{
          "resource": f"//bigquery.googleapis.com/projects/{project_id}/datasets/{data_insights_dataset_name}/tables/{data_insights_data_scan_table_name}"
          }
      }

  """
  {
    "name": "projects/data-analytics-preview/locations/us-central1/operations/operation-1724350067868-62049894382d1-169e9f87-1fe5b7b6",
    "metadata": {
      "@type": "type.googleapis.com/google.cloud.dataplex.v1.OperationMetadata",
      "createTime": "2024-08-22T18:07:50.460547819Z",
      "target": "projects/data-analytics-preview/locations/us-central1/dataScans/chocolate-ai-looker-sales-data-docuemnt-scan",
      "verb": "create",
      "requestedCancellation": false,
      "apiVersion": "v1"
    },
    "done": false
  }
  """

  json_result = restAPIHelper(url, "POST", request_body)

  name = json_result["metadata"]["target"]
  print(f"Document Data Scan created: {name}")
  return name

#### runDataDocumentScan

In [ ]:
def runDataDocumentScan(data_insights_scan_name):
  """Runs the data document scan job and monitors until it completes"""

  """
  !curl --request POST \
    'https://dataplex.googleapis.com/v1/projects/data-analytics-preview/locations/us-central1/dataScans/chocolate-ai-looker-sales-data-docuemnt-scan:run' \
    --header "Authorization: Bearer $(gcloud auth print-access-token)" \
    --header 'Accept: application/json' \
    --header 'Content-Type: application/json' \
    --data '{}' \
    --compressed
  """

  # Create a new scan
  # https://cloud.google.com/dataplex/docs/reference/rest/v1/projects.locations.dataScans/run
  print("Creating Document Data Scan Run")

  url = f"https://dataplex.googleapis.com/v1/projects/{project_id}/locations/{region}/dataScans/{data_insights_scan_name}:run"

  request_body = { }

  """
  {
    "job": {
      "name": "projects/756740881369/locations/us-central1/dataScans/chocolate-ai-looker-sales-data-docuemnt-scan/jobs/27115210-eaf8-43f2-80e8-b2681daa07f0",
      "uid": "27115210-eaf8-43f2-80e8-b2681daa07f0",
      "state": "PENDING",
      "type": "DATA_DOCUMENTATION",
      "createTime": "1970-01-01T00:00:00Z",
      "dataDocumentationSpec": {}
    }
  }
  """

  json_result = restAPIHelper(url, "POST", request_body)
  job_name = json_result["job"]["name"]
  job_state = json_result["job"]["state"]
  print(f"Document Data Scan Run created: {job_name} - State: {job_state}")

  # Monitor the job until it completes
  """
  !curl \
    'https://dataplex.googleapis.com/v1/projects/756740881369/locations/us-central1/dataScans/chocolate-ai-looker-sales-data-docuemnt-scan/jobs/27115210-eaf8-43f2-80e8-b2681daa07f0' \
    --header "Authorization: Bearer $(gcloud auth print-access-token)" \
    --header 'Accept: application/json' \
    --compressed

  {
    "name": "projects/756740881369/locations/us-central1/dataScans/chocolate-ai-looker-sales-data-docuemnt-scan/jobs/27115210-eaf8-43f2-80e8-b2681daa07f0",
    "uid": "27115210-eaf8-43f2-80e8-b2681daa07f0",
    "startTime": "2024-08-22T18:12:17.553066314Z",
    "endTime": "2024-08-22T18:13:06.541798260Z",
    "state": "SUCCEEDED",
    "type": "DATA_DOCUMENTATION",
    "createTime": "2024-08-22T18:12:17.553025391Z"
  }
  STATE_UNSPECIFIED	The DataScanJob state is unspecified.
  RUNNING	The DataScanJob is running.
  CANCELING	The DataScanJob is canceling.
  CANCELLED	The DataScanJob cancellation was successful.
  SUCCEEDED	The DataScanJob completed successfully.
  FAILED	The DataScanJob is no longer running due to an error.
  PENDING	The DataScanJob has been created but not started to run yet.
  """

  url = f"https://dataplex.googleapis.com/v1/{job_name}"
  json_result = restAPIHelper(url, "GET", None)
  while json_result["state"] == "STATE_UNSPECIFIED" or json_result["state"] == "RUNNING" or json_result["state"] == "PENDING":
    print(f"Document Data Scan Run {job_name} - State: {json_result['state']}")
    time.sleep(10)
    json_result = restAPIHelper(url, "GET", None)


#### getDataDocumentScanSql

In [ ]:
def getDataDocumentScanSql(data_insights_scan_name, data_insights_dataset_name,data_insights_data_scan_table_name, data_insight_id_starting_key):
  """Gets the results of the document scan.  If there are no results, then run the job to create the results."""

  """
  !curl \
  'https://dataplex.googleapis.com/v1/projects/data-analytics-preview/locations/us-central1/dataScans/chocolate-ai-looker-sales-data-docuemnt-scan?view=FULL' \
  --header "Authorization: Bearer $(gcloud auth print-access-token)" \
  --header 'Accept: application/json' \
  --compressed

  {
    "name": "projects/data-analytics-preview/locations/us-central1/dataScans/chocolate-ai-looker-sales-data-docuemnt-scan",
    "uid": "4654c390-a841-4939-bd0b-8ba2d151bd03",
    "displayName": "Chocolate A.I. - Looker Sales Data Documenation Scan",
    "state": "ACTIVE",
    "createTime": "2024-08-22T18:07:50.455838572Z",
    "updateTime": "2024-08-22T18:07:55.294013073Z",
    "data": {
      "resource": "//bigquery.googleapis.com/projects/data-analytics-preview/datasets/chocolate_ai/tables/looker_sales_data"
    },
    "executionSpec": {
      "trigger": {
        "onDemand": {}
      }
    },
    "executionStatus": {
      "latestJobStartTime": "2024-08-22T18:12:17.553066314Z",
      "latestJobEndTime": "2024-08-22T18:13:06.541798260Z",
      "latestJobCreateTime": "2024-08-22T18:12:17.553025391Z"
    },
    "type": "DATA_DOCUMENTATION",
    "dataDocumentationSpec": {},
    "dataDocumentationResult": {
      "queries": [
        {
          "sql": "SELECT company_name, item_name, city_name, item_size, customer_name, sale_date, SUM(sale_price) AS total_sale_price FROM `chocolate_ai.looker_sales_data` GROUP BY company_name, item_name, city_name, item_size, customer_name, sale_date;",
          "description": "What is the total sale price of each item name for each company in each city for each item size for each customer name for each sale date?"
        }
      ]
    }
  }
  """

  # First find the cluster if it scan exists
  # https://cloud.google.com/dataplex/docs/reference/rest/v1/projects.locations.dataScans/list

  url = f"https://dataplex.googleapis.com/v1/projects/{project_id}/locations/{region}/dataScans/{data_insights_scan_name}?view=FULL"

  # Gather existing clusters
  json_result = restAPIHelper(url, "GET", None)
  print(f"createDataDocumentScan (GET) json_result: {json_result}")

  # Test to see if cluster exists, if so return
  data_insights_list = []
  if "name" in json_result:
    if "dataDocumentationResult" in json_result:
      for item in json_result["dataDocumentationResult"]["queries"]:
        result_dict = {
            "data_insights_id" : data_insight_id_starting_key,
            "data_insights_scan_name" : data_insights_scan_name,
            "data_insights_dataset_name" : data_insights_dataset_name,
            "data_insights_table_name" : data_insights_table_name,
            "data_insights_sql": item["sql"],
            "data_insights_sql_description": item["description"]
            }
        data_insight_id_starting_key += 1
        data_insights_list.append(result_dict)
    else:
      print("Need to run a job")
      runDataDocumentScan(data_insights_scan_name)
      json_result = restAPIHelper(url, "GET", None)
      if "name" in json_result:
        if "dataDocumentationResult" in json_result:
          for item in json_result["dataDocumentationResult"]["queries"]:
            result_dict = {
                "data_insights_id" : data_insight_id_starting_key,
                "data_insights_scan_name" : data_insights_scan_name,
                "data_insights_dataset_name" : data_insights_dataset_name,
                "data_insights_table_name" : data_insights_table_name,
                "data_insights_sql": item["sql"],
                "data_insights_sql_description": item["description"]
                }
            data_insight_id_starting_key += 1
            data_insights_list.append(result_dict)

  return data_insights_list

#### Create the document scan, the scan job and return the data insights results

In [ ]:
data_insights_scan_name = "looker-sales-data-data-documentation-scan"
data_insights_display_name = "looker_sales_data-data-documentation-scan" # This triggers the BigQuery UI to show the results
data_insights_dataset_name = "chocolate_ai"
data_insights_data_scan_table_name = "looker_sales_data"

data_document_uri = createDataDocumentScan(data_insights_scan_name, data_insights_display_name, data_insights_dataset_name, data_insights_data_scan_table_name)

In [ ]:
data_insights_table_name = "data_insights"
data_insights_table_primary_key = "data_insights_id"

data_insight_id_starting_key = GetNextPrimaryKey(f"{project_id}.{data_insights_dataset_name}.{data_insights_table_name}", data_insights_table_primary_key)

data_insights_list = getDataDocumentScanSql(data_insights_scan_name, data_insights_dataset_name, data_insights_table_name, data_insight_id_starting_key)

#### Save results to BigQuery

In [ ]:
import pandas as pd
from google.cloud import bigquery

bigquery_client = bigquery.Client()

# Bulk insert the results
table_id = f"{project_id}.chocolate_ai.data_insights"

dataframe = pd.DataFrame(
    pd.DataFrame(data_insights_list), # Your source data
    columns=[
        "data_insights_id",
        "data_insights_scan_name",
        "data_insights_dataset_name",
        "data_insights_table_name",
        "data_insights_sql",
        "data_insights_sql_description",
    ],
)

job_config = bigquery.LoadJobConfig(
    schema=[
        bigquery.SchemaField("data_insights_id", bigquery.enums.SqlTypeNames.INT64, mode="REQUIRED"),
        bigquery.SchemaField("data_insights_scan_name", bigquery.enums.SqlTypeNames.STRING, mode="REQUIRED"),
        bigquery.SchemaField("data_insights_dataset_name", bigquery.enums.SqlTypeNames.STRING, mode="REQUIRED"),
        bigquery.SchemaField("data_insights_table_name", bigquery.enums.SqlTypeNames.STRING, mode="REQUIRED"),
        bigquery.SchemaField("data_insights_sql", bigquery.enums.SqlTypeNames.STRING, mode="REQUIRED"),
        bigquery.SchemaField("data_insights_sql_description", bigquery.enums.SqlTypeNames.STRING, mode="REQUIRED"),
    ],
    write_disposition="WRITE_APPEND",
)

job = bigquery_client.load_table_from_dataframe(dataframe, table_id, job_config=job_config)
job.result()  # Wait for the job to complete.

table = bigquery_client.get_table(table_id)  # Make an API request.
print("Loaded {} rows and {} columns to {}".format(table.num_rows, len(table.schema), table_id))

In [ ]:
%%bigquery

SELECT * FROM `chocolate_ai.data_insights` ORDER BY data_insights_id;

#### Update BigQuery (Console) to see these Insights

##### updateBigQueryTableInsightsLabels

In [ ]:
def updateBigQueryTableInsightsLabels(data_insights_scan_id, data_insights_dataset_name, data_insights_data_scan_table_name):
  """Sets the labels on the BigQuery table so users can see the data insights in the Console."""

  """
    curl --request PATCH \
        "https://bigquery.googleapis.com/bigquery/v2/projects/{local_project_id}/datasets/{local_dataset_name}/tables/{local_table_name}" \
        --header "Authorization: Bearer $token" \
        --header "Accept: application/json" \
        --header "Content-Type: application/json" \
        --data "{
          \"labels\":
          {\"dataplex-dp-published-location\":\"{local_dataplex_region}\",
          \"dataplex-dp-published-project\":\"{local_project_id}\",
          \"dataplex-dp-published-scan\":\"{local_scan_name}\"}}" \
        --compressed
  """

  # Patch BigQuery
  # https://cloud.google.com/dataplex/docs/reference/rest/v1/projects.locations.dataScans/create
  print("Patching BigQuery Data Insights Labels")

  url = f"https://bigquery.googleapis.com/bigquery/v2/projects/{project_id}/datasets/{data_insights_dataset_name}/tables/{data_insights_data_scan_table_name}"

  request_body = {
      "labels" : {
          "dataplex-data-documentation-published-location" : region,
          "dataplex-data-documentation-published-project" : project_id,
          "dataplex-data-documentation-published-scan" : data_insights_scan_id,
          }
      }

  json_result = restAPIHelper(url, "PATCH", request_body)
  print(json_result)

##### Patch the BigQuery table so the Data Insights show in the UI

In [ ]:
# This will tell the BigQuery UI about this data document scan and the results will also show in the BQ UI
updateBigQueryTableInsightsLabels(data_insights_scan_name, data_insights_dataset_name, data_insights_data_scan_table_name)

### Clean Up

In [ ]:
# Placeholder

### Reference Links


- [Google.com](https://www.google.com)